In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.\
config('spark.shuffle.useOldFetchProtocol', 'true').\
enableHiveSupport().\
appName("Project_Pyspark").\
master('yarn').\
getOrCreate()

In [3]:
spark

In [4]:
raw_df=spark.read.format("csv").option("header","true").\
option("header","true").\
load("/public/trendytech/datasets/accepted_2007_to_2018Q4.csv") #reading 1.7GB file

In [5]:
raw_df.createOrReplaceTempView("Lending_Club_Table") #converted to table for better view

In [6]:
spark.sql("select * from Lending_Club_Table limit 2") #to check the data 

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
68407277,null,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingcl...,null,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,null,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723916800001,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,null,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,null,null,null,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,null,null,null,null,null,null,null,null,null,null,null,null,null,N,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Cash,N,null,null,null,null,null,null
68355089,null,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingcl...,null,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,null,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.66,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,null,Mar-2019,699.0,695.0,0.0,null,1.0,Individual,null,null,null,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,null,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,null,null,null,null,null,null,null,null

1  1.7GB file we need to split this into multiple files for utilizing size efficiently,their are 116 cols used 

2  we can see from above member_id is the person who took loan but here their are nulls present,
  we need to create this member_id by applying logic.
  
3 id column is Loan_id there can be multiple loan_id per member_id since single person can take multiple loans.

4 CSV format available, 2+ million rows.

5 We can create 4 datasets out of 1 big file i.e; 1) customers data (borrowers) 2) loans data 3) loan_repayments 4) loan_defaulters

according we can divide 116 cols into 4 differents datasets.

We need "member_id" since its a null value we will creating a newcolumn called "sha2"its a hash value with length of 32 characters, this acts as a member_id from combination/concat of multiple columns the columns would be emp_title, emp_length, home_ownership, annual_inc, zip_code, addr_state, grade, sub_grade, verification_status. If these columns are repeated then sha2 column will also get repeated. 


In [7]:
from pyspark.sql.functions import sha2,concat_ws

In [8]:
new_df=raw_df.withColumn("name_sha2",sha2(concat_ws("||",*["emp_title","emp_length","home_ownership","annual_inc","zip_code","addr_state","grade","sub_grade",\
                  "verification_status"]),256))

In [9]:
new_df.select("name_sha2").show(1,False) #we can see a new column is generated 

+----------------------------------------------------------------+
|name_sha2                                                       |
+----------------------------------------------------------------+
|6d5091b3fcaaeb4eac37445042b6b79e8b21f16943b9c32c4f5dd74dcb0f2210|
+----------------------------------------------------------------+
only showing top 1 row



In [10]:
new_df.createOrReplaceTempView("newtable") #creating a newtable to query it 

In [11]:
spark.sql("select  count(*) as Total_Count from newtable") #checking count we can seee there are 2.2+ million records, now we need to check if new column generated is unique or
#any duplicates are present

Total_Count
2260701


In [12]:
spark.sql("select count(distinct(name_sha2)) members_taken_loan from newtable") #we can see there is mismatch in count stating around 3k people have taken \
#multiple loans or duplicate member_ids present 

members_taken_loan
2257384


In [18]:
spark.sql("""select name_sha2,count(*) as total_count from newtable 
             group by name_sha2 
             having total_count >1
             order by total_count desc
             limit 10
""") #we can see the first one looks like outlier bcz loan was given 33 times this isn't practically possible lets explore more on this record.

name_sha2,total_count
e3b0c44298fc1c149...,33
e4c167053d5418230...,5
76b577467eda5bdbc...,4
ad8e5d384dae17e06...,4
3f87585a20f702838...,4
059d401bb603d9a80...,3
819453be77718d747...,3
c92062bb371842b3d...,3
4a2abc8996196b17e...,3
f42447e25d558b419...,3


In [19]:
spark.sql("select * from newtable where name_sha2 LIKE 'e3b0c44298fc1c149%' ") 
#we can see this is repeating 33 times so the records are more and even all are nulls making this as exception we can drop this  

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,name_sha2
Total amount fund...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,e3b0c44298fc1c149...
Total amount fund...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,e3b0c

1 Since we are creating 4 different files from 1.7 GB file thus we need to separate these cols as well as per the filename mentioned

1) customers data (borrowers) 
 
  member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_limit,
  application_type,annual_inc_joint,verification_status_joint

2) loans data 
  
  loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title

3) loan_repayments 
  
  loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d

4) loan_defaulters
  
  member_id,delinq_2yrs,delinq_amnt,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record
  
2 Creating a separate file for these cols below
  

In [20]:
spark.sql("""select 
name_sha2 as member_id, emp_title, emp_length, home_ownership, annual_inc, addr_state, zip_code,
'USA' as country, grade, sub_grade, verification_status, tot_hi_cred_lim,
application_type, annual_inc_joint, verification_status_joint
from newtable
      """).repartition(1).write.option("header",True)\
.format("csv").mode("overwrite").option("path","/user/itv012010/lendingclubproject/raw/customers_data_csv").save() #savings file in csv format 

#while making a single partition we can make use of coalesce as well but since we are drastically reducing cols so repartition is recommended 

In [21]:
customers_df=spark.read.format("csv").option("header",True).load("/user/itv012010/lendingclubproject/raw/customers_data_csv").show(1) #we can see data is present

+--------------------+------------------+----------+--------------+----------+----------+--------+-------+-----+---------+-------------------+---------------+----------------+----------------+-------------------------+
|           member_id|         emp_title|emp_length|home_ownership|annual_inc|addr_state|zip_code|country|grade|sub_grade|verification_status|tot_hi_cred_lim|application_type|annual_inc_joint|verification_status_joint|
+--------------------+------------------+----------+--------------+----------+----------+--------+-------+-----+---------+-------------------+---------------+----------------+----------------+-------------------------+
|a4ec00ba67fadf2fe...|Executive Director|  < 1 year|      MORTGAGE|   81000.0|        PA|   152xx|    USA|    C|       C5|           Verified|       229529.0|      Individual|            null|                     null|
+--------------------+------------------+----------+--------------+----------+----------+--------+-------+-----+---------+--

In [22]:
spark.sql("""select 
id as loan_id, name_sha2 as member_id, loan_amnt, funded_amnt, term, int_rate, installment, issue_d, loan_status, purpose, title
     from newtable  """).repartition(1).write.option("header",True)\
.format("csv").mode("overwrite").option("path","/user/itv012010/lendingclubproject/raw/loans_data_csv").save() #savings file in csv format 

#while making a single partition we can make use of coalesce as well but since we are drastically reducing cols so repartition is recommended 

In [75]:
loan_df=spark.read.format("csv").option("header",True).load("/user/itv012010/lendingclubproject/raw/loans_data_csv").show(1) #loan df is ready

+--------+--------------------+---------+-----------+---------+--------+-----------+--------+-----------+------------------+------------------+
| loan_id|           member_id|loan_amnt|funded_amnt|     term|int_rate|installment| issue_d|loan_status|           purpose|             title|
+--------+--------------------+---------+-----------+---------+--------+-----------+--------+-----------+------------------+------------------+
|94761152|1ae08eb0c074de6fb...|  26975.0|    26975.0|60 months|   30.84|     886.71|Dec-2016| Fully Paid|debt_consolidation|Debt consolidation|
+--------+--------------------+---------+-----------+---------+--------+-----------+--------+-----------+------------------+------------------+
only showing top 1 row



In [128]:
spark.sql("""select id as loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d
from newtable
      """).repartition(1).write\
.option("header",True).format("csv").mode("overwrite").option("path","/user/itv012010/lendingclubproject/raw/loans_repayment_csv").save() #saving it 

In [129]:
loans_repayment_df=spark.read.format("csv").option("header",True).load("/user/itv012010/lendingclubproject/raw/loans_repayment_csv").show(1) #data is ready 

+--------+---------------+-------------+------------------+-----------+---------------+------------+------------+
| loan_id|total_rec_prncp|total_rec_int|total_rec_late_fee|total_pymnt|last_pymnt_amnt|last_pymnt_d|next_pymnt_d|
+--------+---------------+-------------+------------------+-----------+---------------+------------+------------+
|69642551|        3246.03|       990.53|               0.0|    4965.02|         326.35|    Feb-2017|        null|
+--------+---------------+-------------+------------------+-----------+---------------+------------+------------+
only showing top 1 row



In [23]:
spark.sql("""select name_sha2 as member_id, delinq_2yrs, delinq_amnt, pub_rec,pub_rec_bankruptcies, inq_last_6mths, total_rec_late_fee, 
mths_since_last_delinq, mths_since_last_record
from newtable
      """).repartition(1).write\
.mode("overwrite").format("csv").option("header",True).option("path","/user/itv012010/lendingclubproject/raw/loans_defaulters_csv").save() #saving it 

In [137]:
loans_defaulters_df=spark.read.format("csv").option("header",True).load("/user/itv012010/lendingclubproject/raw/loans_defaulters_csv").show(1)

+--------------------+-----------+-----------+--------------------+--------------+------------------+----------------------+----------------------+
|           member_id|delinq_2yrs|delinq_amnt|pub_rec_bankruptcies|inq_last_6mths|total_rec_late_fee|mths_since_last_delinq|mths_since_last_record|
+--------------------+-----------+-----------+--------------------+--------------+------------------+----------------------+----------------------+
|2209556c19f75a204...|        0.0|        0.0|                 1.0|           0.0|               0.0|                  null|                  65.0|
+--------------------+-----------+-----------+--------------------+--------------+------------------+----------------------+----------------------+
only showing top 1 row



Cleaning of Data is required

For Customers_data

1. Create a dataframe with proper datatypes (modify schema)
2. Rename a few columns 
   annual_inc -> annual_income
   addr_state -> address_state
   zip_cod    -> address_zipcode
   country    -> address_country
   tot_hi_cred_lim -> total_high_credit_limit
   annual_inc_joint -> join_annual_income
3. insert a new column names ingestion date (current time)
4. remove complete duplicate rows
5. remove rows where annual_income is null
6. convert emp_length to integer
7. we need to replace all the nulls in emp_length column with average of this column
8. clean the address_state (it should be 2 characters only), replace all others with NA

write the cleaned data in cleaned folder in HDFS 

In [125]:
customers_raw_df=spark.read.format("csv").option("inferSchema",True).option("header",True).load("/user/itv012010/lendingclubproject/raw/customers_data_csv")

In [126]:
customers_raw_df.printSchema() #we can see annual_inc, tot_hi_cred_lim,annual_inc_joint these needs to be float 

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- tot_hi_cred_lim: double (nullable = true)
 |-- application_type: string (nullable = true)
 |-- annual_inc_joint: string (nullable = true)
 |-- verification_status_joint: string (nullable = true)



In [4]:
Customers_schema = "member_id string, emp_title string, emp_length string,home_ownership string,annual_inc float,addr_state string,zip_code string,\
                    country string,grade string,sub_grade string,verification_status string,tot_hi_cred_lim float,application_type string,\
                    annual_inc_joint string,verification_status_joint float"

In [5]:
customers_raw_df=spark.read.format("csv").schema(Customers_schema).option("header",True).load("/user/itv012010/lendingclubproject/raw/customers_data_csv") 
#removing inferSchema

In [6]:
customers_raw_df.printSchema() #now the schema is changed

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- tot_hi_cred_lim: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- annual_inc_joint: string (nullable = true)
 |-- verification_status_joint: float (nullable = true)



In [7]:
customers_df_renamed=customers_raw_df.withColumnRenamed("annual_inc","annual_income")\
.withColumnRenamed("addr_state","address_state")\
.withColumnRenamed("zip_cod","address_zipcode")\
.withColumnRenamed("country","address_country")\
.withColumnRenamed("tot_hi_cred_lim","total_high_credit_limit")\
.withColumnRenamed("annual_inc_joint","join_annual_income")\
#renaming col done

In [8]:
from pyspark.sql.functions import *

In [9]:
customers_df_ingested=customers_df_renamed.withColumn("ingest_date",current_timestamp()) #new col with current timestamp 

In [10]:
customers_df_ingested.select("ingest_date").show(1,False) #we can see date is added 

+-----------------------+
|ingest_date            |
+-----------------------+
|2024-03-26 05:26:17.477|
+-----------------------+
only showing top 1 row



In [11]:
customers_df_ingested.count() #total count 

2260701

In [12]:
customers_df_distinct=customers_df_ingested.distinct()

In [13]:
customers_df_distinct.count() #we get distinct rows after removing duplicates

2260638

In [17]:
customers_df_distinct.createOrReplaceTempView("customers") #converting to table 

In [18]:
spark.sql("select count(*) from customers where annual_income is null") #we need to delete these 5 rows 

count(1)
5


In [19]:
customers_income_filtered=spark.sql("select * from customers where annual_income is not null") #to create another table with no null 

In [20]:
customers_income_filtered.createOrReplaceTempView("customers") #this table customers is updated with latest no null values from above query

In [21]:
spark.sql("select distinct(emp_length) from customers").show() #we need to convert this to integer only numbers to be present

+----------+
|emp_length|
+----------+
|   5 years|
|   9 years|
|      null|
|    1 year|
|   2 years|
|   7 years|
|   8 years|
|   4 years|
|   6 years|
|   3 years|
| 10+ years|
|  < 1 year|
+----------+



In [22]:
from pyspark.sql.functions import regexp_replace, col

In [23]:
customers_emp_length_cleaned=customers_income_filtered.withColumn("emp_length",regexp_replace(col("emp_length"),"(\D)",""))#this replaces non digit value with spaces \D is for non digit value

In [24]:
customers_emp_length_cleaned.printSchema() #we can see still emp_length is string we need to convert type to int

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_income: float (nullable = true)
 |-- address_state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- address_country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- total_high_credit_limit: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- join_annual_income: string (nullable = true)
 |-- verification_status_joint: float (nullable = true)
 |-- ingest_date: timestamp (nullable = false)



In [25]:
customers_emp_length_casted=customers_emp_length_cleaned.withColumn("emp_length",customers_emp_length_cleaned.emp_length.cast("int")) #casting datatype to int

In [26]:
customers_emp_length_casted.printSchema() #now the datatype is changed to int

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: integer (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_income: float (nullable = true)
 |-- address_state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- address_country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- total_high_credit_limit: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- join_annual_income: string (nullable = true)
 |-- verification_status_joint: float (nullable = true)
 |-- ingest_date: timestamp (nullable = false)



In [27]:
customers_emp_length_casted.filter("emp_length is null").count() #total nulls within this col

146903

In [28]:
customers_emp_length_casted.createOrReplaceTempView("customers") #creating a table on top of it

In [29]:
avg_emp_length=spark.sql("select floor(avg(emp_length)) as avg_emp_length from customers").collect() #avg value 

In [30]:
print(avg_emp_length) 

[Row(avg_emp_length=6)]


In [31]:
avg_emp_duration=avg_emp_length[0][0] #we want a number to be replaced with all nulls thus making it look via 2D matrix 

In [32]:
print(avg_emp_duration) 

6


In [33]:
customers_emp_length_replaced=customers_emp_length_casted.na.fill(avg_emp_duration,subset=['emp_length']) 
#to fill nulls with avg(emp_length) in place of nulls present in emp_length col

In [34]:
customers_emp_length_replaced.filter('emp_length is null').count()#no nulls are present now

0

In [35]:
customers_emp_length_replaced.createOrReplaceTempView('customers')

In [44]:
spark.sql("select address_state from customers where length(address_state)>2") #we can see there are many address_state >2 in length

address_state
I have never owne...
debt_consolidation
we do consider ou...
and I want to get...
Advocate business...
wholesale and eve...
941xx
013xx
594xx
Tax Loan


In [45]:
spark.sql("select count(address_state) from customers where length(address_state)>2") #count of them 

count(address_state)
254


In [46]:
from pyspark.sql.functions import length

In [52]:
customers_state_cleaned=customers_emp_length_replaced.withColumn \
("address_state", \
 when(length(col("address_state"))>2,"NA").otherwise(col("address_state")) \
) #this will replace address_state>2 to NA and keep others as same address_state

In [55]:
customers_state_cleaned.select("address_state").distinct() #we can see all address_state is <=2 as expected.

address_state
AZ
SC
LA
MN
NJ
DC
OR
NA
VA
null


In [56]:
customers_state_cleaned.write\
.format("parquet")\
.mode("overwrite")\
.option("path","/user/itv012010/lendingclubproject/cleaned/customers_parquet")\
.save() #Finally saving this cleaned data in parquet format

In [57]:
customers_state_cleaned.write\
.option("header",True)\
.format("csv")\
.mode("overwrite")\
.option("path","/user/itv012010/lendingclubproject/cleaned/customers_csv")\
.save() #Finally saving this cleaned data in csv format also.